## Calculate GAM in metabolic model
    **Version:** 1.0
    **Author:** Shakra Ahmad
    ** Modified by:** Sol Han
    **Organization:** Sybirg Lab
    **Description:** This code is used to calculate GAM in metabolic model.

    Copyright (c) 2023 Sybirg Lab, Konkuk University 
    All rights reserved. 

    **Note: This code is provided as-is, without any warranties or guarantees.*

In [ ]:
# metabolic model open
import cobra

s12 = cobra.io.read_sbml_model('iSH1474_GAM_NGAM.xml')
model = s12.copy()

In [ ]:
# Change the coefficients of amino acids and dnas in biomass equations

aa = ['ala__L_c','arg__L_c','asn__L_c','asp__L_c','cys__L_c','gln__L_c','glu__L_c','gly_c','his__L_c','ile__L_c',
       'leu__L_c','lys__L_c','met__L_c','phe__L_c','pro__L_c','ser__L_c','thr__L_c','trp__L_c','tyr__L_c','val__L_c']
dna = ['datp_c','dctp_c','dgtp_c','dttp_c']

aa_coeffs = {'ala__L_c':-0.609649,'arg__L_c':-0.363607,'asn__L_c':-0.160967,'asp__L_c':-0.291447,'cys__L_c':-0.058177,
          'gln__L_c':-0.257130,'glu__L_c':-0.310237,'gly_c':-0.436021,'his__L_c':-0.129586,'ile__L_c':-0.249629,
       'leu__L_c':-0.644223,'lys__L_c':-0.185472,'met__L_c':-0.127920,'phe__L_c':-0.194990,'pro__L_c':-0.267482,
          'ser__L_c':-0.310657,'thr__L_c':-0.259012,'trp__L_c':-0.079935,'tyr__L_c':-0.139980,'val__L_c':-0.393654}

dna_coeffs = {'datp_c':-0.010765,'dctp_c':-0.017418,'dgtp_c':-0.017382,'dttp_c':-0.010745}

biomass_equations = ['BIOMASS_KT2440_Core2','BIOMASS_KT2440_WT3']

for eq in biomass_equations:
    
    delete = {}

    for m in aa+dna:
        globals()[m] = model.metabolites.get_by_id(m)
        globals()[m+'_coef'] = model.reactions.get_by_id(eq).get_coefficient(globals()[m])
        delete[m]=-(globals()[m+'_coef'])
    
    model.reactions.get_by_id(eq).add_metabolites(delete)
    model.reactions.get_by_id(eq).add_metabolites(aa_coeffs)
    model.reactions.get_by_id(eq).add_metabolites(dna_coeffs)
    
    print(eq, model.reactions.get_by_id(eq).reaction)

In [ ]:
# q= growth_rate/Y_G + maintenance coefficient(mc)

inverse_Y_G = 10
mc = 0.077



# NGAM calculation

import numpy as np

biomass_equations = ['BIOMASS_KT2440_Core2','BIOMASS_KT2440_WT3']

for eq in biomass_equations:
    for m in gam_am:
        globals()[m] = model.metabolites.get_by_id(m)
        globals()[m+'_coef'] = model.reactions.get_by_id(eq).get_coefficient(globals()[m])

    with model:
        model.reactions.get_by_id('ATPM').bounds = (-99999,99999)
        model.reactions.get_by_id(eq).add_metabolites({'h2o_c':-h2o_c_coef,'atp_c':-atp_c_coef,'adp_c':-adp_c_coef,'h_c':-h_c_coef,'pi_c':-pi_c_coef})
    
        model.objective = 'ATPM'
        model.reactions.get_by_id("EX_glc__D_e").lower_bound = -mc
        model.reactions.get_by_id('BIOMASS_KT2440_Core2').lower_bound = 0
        model.reactions.get_by_id('BIOMASS_KT2440_WT3').lower_bound = 0
        ngam = round(model.optimize().objective_value,2)
        
print('NGAM:',ngam)

In [ ]:
# GAM calculation

glucose = [1,2,3,4,5] #mmol

met = ['h2o_c','atp_c','adp_c','h_c','pi_c'] # gam associated metabolites
biomass = model.reactions.get_by_id('BIOMASS_KT2440_Core2')
model.objective = 'BIOMASS_KT2440_Core2'

for m in met:
    globals()[m] = model.metabolites.get_by_id(m)
    globals()[m+'_coef'] = biomass.get_coefficient(globals()[m])

for coef in (np.arange(40,50,0.01)):
    objective_values = []
    
    with model:
        model.reactions.get_by_id('ATPM').bounds = (ngam,ngam)
        for glc in glucose:
            model.reactions.get_by_id("EX_glc__D_e").lower_bound = -glc
            biomass.add_metabolites({'h2o_c':-(coef-5.4698)-h2o_c_coef,'atp_c':-(coef+0.168683)-atp_c_coef,'adp_c':coef-adp_c_coef,'h_c':coef-h_c_coef,'pi_c':coef-pi_c_coef-0.003522})
            objective_values.append(model.optimize().objective_value)
            
        slope, intercept = np.polyfit(objective_values, glucose,1)
        
        print('gam:',coef, 'slope:',slope)

In [ ]:
#fix the GAM/NGAM

gam = 45.21

for eq in biomass_equations:
    for m in met:
        globals()[m] = model.metabolites.get_by_id(m)
        globals()[m+'_coef'] = model.reactions.get_by_id(eq).get_coefficient(globals()[m])
    model.reactions.get_by_id(eq).add_metabolites({'h2o_c':-h2o_c_coef,'atp_c':-atp_c_coef,'adp_c':-adp_c_coef,'h_c':-h_c_coef,'pi_c':-pi_c_coef})

    model.reactions.get_by_id(eq).add_metabolites({'h2o_c':round(-(gam-5.4698),6),
                                                                 'atp_c':round(-(gam+0.168683),6),
                                                                 'adp_c':round(gam,6),
                                                                 'h_c':round(gam,6),
                                                                 'pi_c':round(gam-0.003522,6)})
    print(model.reactions.get_by_id(eq).get_coefficient(adp_c))
    
model.reactions.get_by_id('ATPM').bounds = (ngam, 999999.0)


# save the model

model.objective = 'BIOMASS_KT2440_WT3'

cobra.io.write_sbml_model(model, 'iSH1474_GAM_NGAM.xml')
cobra.io.save_matlab_model(model, 'iSH1474_GAM_NGAM.mat')
cobra.io.save_json_model(model, 'iSH1474_GAM_NGAM.json')